In [15]:
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms


In [16]:
def verify_images(image_folder):
    classes = ["Cat", "Dog"]
    class_to_idx = {"Cat": 0, "Dog": 1}
    samples = []
    for cls_name in classes:
        cls_dir = os.path.join(image_folder, cls_name)
        for fname in os.listdir(cls_dir):
            if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            path = os.path.join(cls_dir, fname)
            try:
                with Image.open(path) as img:
                    img.verify()
                samples.append((path, class_to_idx[cls_name]))
            except Exception:
                print(f"Warning: Skipping corrupted image {path}")
    
    return samples

In [17]:
class ImageDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        with Image.open(path) as img:
            img = img.convert('RGB')
            if self.transform:
                img = self.transform(img)
        
        return img, label

In [18]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=1, kernel_size=1),   # 1乘1卷积
            nn.AdaptiveAvgPool2d((1, 1)),  # 全局平均池化层
            nn.Flatten(),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [19]:
DATA_DIR = r"/Users/x/Rust/DeepLearning/PetImages"
BATCH_SIZE = 32
IMG_SIZE = 128
EPOCHS = 10
LR = 0.001
PRINT_STEP = 100

DEVICE = torch.device("mps" if torch.mps.is_available() else "cpu")

In [20]:
def evaluate(model, test_dataloader):
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(DEVICE)
            labels = labels.float().unsqueeze(1).to(DEVICE)

            outputs = model(inputs)
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total;
    return val_acc

In [21]:
all_samples = verify_images(DATA_DIR)

In [22]:
random.seed(42)
random.shuffle(all_samples)
train_size = int(len(all_samples) * 0.8)
train_samples = all_samples[:train_size]
valid_samples = all_samples[train_size:]

In [23]:
data_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [24]:
train_dataset = ImageDataset(train_samples, data_transform)
valid_dataset = ImageDataset(valid_samples, data_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [25]:
model = CNNModel().to(DEVICE)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [26]:
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    model.train()
    running_loss = 0.0

    for step, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (step + 1) % PRINT_STEP == 0:
            avg_loss = running_loss / PRINT_STEP
            print(f".   Step [{step + 1}] - Loss: {avg_loss:.4f}")
            running_loss = 0.0
        
    val_acc = evaluate(model, valid_dataloader)
    print(f"Validation Accuray after epoch {epoch + 1}: {val_acc:.4f}")


Epoch 1/10
.   Step [100] - Loss: 0.6773
.   Step [200] - Loss: 0.6504
.   Step [300] - Loss: 0.6542
.   Step [400] - Loss: 0.6344
.   Step [500] - Loss: 0.6384
.   Step [600] - Loss: 0.6127
Validation Accuray after epoch 1: 0.6766

Epoch 2/10
.   Step [100] - Loss: 0.5974
.   Step [200] - Loss: 0.5949
.   Step [300] - Loss: 0.5704
.   Step [400] - Loss: 0.5717
.   Step [500] - Loss: 0.5756
.   Step [600] - Loss: 0.5448
Validation Accuray after epoch 2: 0.7110

Epoch 3/10
.   Step [100] - Loss: 0.5568
.   Step [200] - Loss: 0.5273
.   Step [300] - Loss: 0.5348
.   Step [400] - Loss: 0.5261
.   Step [500] - Loss: 0.5287
.   Step [600] - Loss: 0.4996
Validation Accuray after epoch 3: 0.7276

Epoch 4/10
.   Step [100] - Loss: 0.4901
.   Step [200] - Loss: 0.5003
.   Step [300] - Loss: 0.4626
.   Step [400] - Loss: 0.4850
.   Step [500] - Loss: 0.4693
.   Step [600] - Loss: 0.4658
Validation Accuray after epoch 4: 0.7876

Epoch 5/10
.   Step [100] - Loss: 0.4451
.   Step [200] - Loss: 0.4